# Import

In [1]:
import pandas as pd

## Get Excel File

In [16]:
# path = input('\nInsert excel-path here: ')
path = r'"C:\Users\eilon.eilstein\Python\Excel Validation for Snir\2021-07-29-summary.xlsx"'

if path[0] in ['"', "'"]:
    path = path[1:-1]

# Slice Excel's Name and Date
slash_i = len(path) - path[::-1].find('\\')
dot_i = len(path) - path[::-1].find('.')
XL_name = path[slash_i:dot_i-1]
XL_date = XL_name[:10]

XL_dict = pd.read_excel(path, sheet_name=None)
xl_sheets = list(XL_dict.keys())
xl_sheets


['סיכום', 'כללית', 'מכבי', 'מאוחדת', 'לאומית', 'צהל', 'נובולוג', 'מלאי שסופק']

## Get Snir's Summary Sheet

In [17]:
df_sum_snir = pd.read_excel(path, sheet_name=xl_sheets[0])
df_sum_snir_cols = df_sum_snir.columns
df_sum_snir = df_sum_snir[df_sum_snir_cols[:12]]
df_sum_snir.head()

,לקוח,נקודות הפצה,"סה""כ ויאלים",מגשי 10,מגשי 5,מגשי 1,"סה""כ חיסונים",מנה א',מנה ב',אלגוריתם מכירה,Unnamed: 10,Unnamed: 11
0,כללית,69,1010,101.0,0.0,0.0,10100.0,1010.0,0.0,מגש נוסף עבור מכירה,261.0,סך ויאלים/195
1,מכבי,47,500,50.0,0.0,0.0,5000.0,500.0,0.0,מספר מגשים חדש,261.0,"סה""כ למכירה"
2,מאוחדת,60,760,76.0,0.0,0.0,7600.0,570.0,190.0,במידה ולא מצליחים למכור-מספר ויאלים לביטול:,NaN,NaN
3,לאומית,16,160,16.0,0.0,0.0,1600.0,160.0,0.0,NaN,NaN,NaN
4,נובולוג,0,170,17.0,0.0,0.0,1700.0,0.0,170.0,NaN,NaN,NaN


## Set important variables

In [18]:
tray_dict = {1  : 'PFIZER-1VIAL PACKAGE-COVID-19 VACCINE29G'
           , 5  : 'PFIZER- 5 PACKAGE-COVID-19 VACCINE 29G'
           , 10 : 'COVID VACCINE-10 Vial/Box'
           , 15 : 'COVID VACCINE-15 Vial/Box'
           , 50 : 'COVID VACCINE-50 Vial/Box'
           , 195: 'חיסון קורונה - מגש 195'
           , 1  : 'MODERNA COVID-19 m-RNA1273 VAC 5ML 1VIAL'
           , 5  : 'MODERNA COVID-19 m-RNA1273 VAC 5ML 5VIAL'
           , 10 : 'MODERNA COVID19 MRNA1273 VACCINE 5ML 29C 10VIAL'}

tray_dict = {  'PFIZER-1VIAL PACKAGE-COVID-19 VACCINE29G' : [1, 'P']
             , 'PFIZER- 5 PACKAGE-COVID-19 VACCINE 29G' : [5, 'P']
             , 'COVID VACCINE-10 Vial/Box' : [10, 'P']
             , 'COVID VACCINE-15 Vial/Box' : [15, 'P']
             , 'COVID VACCINE-50 Vial/Box' : [50, 'P']
             , 'חיסון קורונה - מגש 195' : [195, 'P']
             , 'MODERNA COVID-19 m-RNA1273 VAC 5ML 1VIAL' : [1, 'M']
             , 'MODERNA COVID-19 m-RNA1273 VAC 5ML 5VIAL' : [5, 'M']
             , 'MODERNA COVID19 MRNA1273 VACCINE 5ML 29C 10VIAL' : [10, 'M']}

vial_to_vaccs_dict = {'M' : 10, 'P' : 6}

reversed_tray_dict = {val[0]: key for (key, val) in tray_dict.items()}

relevant_cols = ['תיאור פריט', 'קוד לקוח', 'סה"כ כמות מארזים']

tray_keys = list(reversed_tray_dict.keys())[::-1]

final_cols = ['Entity', 'distribution points', 'Sum Vials', 'full trays', 'splitted trays'] + tray_keys[1:] + ['Sum vials splitted', 'Sum vaccinations', 'isMissing' ]

vial_to_vaccs_dict

{'M': 10, 'P': 6}

## Define Validation Function

In [19]:
def validate_sheet(sheet_name):
    # sheet_name = 'כללית'
    # def validate_sheet(sheet_name):
    # sheet_name = 'כללית'

    # Get Excel sheet
    df_hosp = pd.read_excel(path, sheet_name=sheet_name)
    df_hosp
    # Check if sheet is empty
    if df_hosp.shape == (0, 0):
        return None ############
#             pass

    # Calculate number of distribution points
    POINTS = len(df_hosp[relevant_cols[1]].unique())
    df_hosp[relevant_cols[1]] = df_hosp[relevant_cols[1]].apply(lambda x: str(x))

    # Filter on relevant products
    df_hosp_filtered = df_hosp[df_hosp[relevant_cols[0]].isin(tray_dict.keys())][relevant_cols]

    # Change category name to numeric
    # df_hosp_filtered[relevant_cols[0]] = df_hosp_filtered[relevant_cols[0]].apply(lambda x: tray_dict[x][0])
    # df_hosp_filtered.sort_values([relevant_cols[0]], inplace=True)


    # fill missing categories
    all_vals = list(tray_dict.keys()) # get all possible values
    sheet_vals = df_hosp_filtered[relevant_cols[0]].unique() # get all existing values in sheet
    lst_addition = []

        # check for missing values
    for i in all_vals:
        if i not in list(sheet_vals):
            lst_addition.append([i, 0, 0])

        # append fillers
    df_hosp_filtered = pd.concat([df_hosp_filtered, pd.DataFrame(lst_addition, columns=df_hosp_filtered.columns)])


    # Add column - Manufacturer
    df_hosp_filtered['Manufacturer'] = df_hosp_filtered[relevant_cols[0]].apply(lambda x: tray_dict[x][1])
    df_hosp_filtered['VialsPerTray'] = df_hosp_filtered[relevant_cols[0]].apply(lambda x: tray_dict[x][0])
    add_cols = ['VialsPerTray', 'Manufacturer']


    # # Group df by product name
    df_grouped = df_hosp_filtered.groupby(add_cols).sum().reset_index()
    df_grouped['Vials'] = df_grouped[relevant_cols[2]] * df_grouped[add_cols[0]]
    # df_grouped = df_grouped.T.rename(columns=reversed_tray_dict) # pivot



    df_vaccines = df_grouped.groupby(add_cols[1]).sum().reset_index()

    df_vaccines['Vaccines'] = df_vaccines[add_cols[1]].apply(lambda x: vial_to_vaccs_dict[x]) * df_vaccines['Vials']

    sum_vaccs = df_vaccines.Vaccines.sum()

    df_grouped = df_grouped.groupby('VialsPerTray').sum()

    sum_vials = df_grouped['Vials'].sum()

    df_grouped = df_grouped.T.reset_index() # pivot
    df_grouped = df_grouped[df_grouped['index'] == relevant_cols[-1]]

    df_grouped['distribution points'] = POINTS
    df_grouped['Entity'] = sheet_name

    sum_trays = df_vaccines[relevant_cols[-1]].sum()
    # df_vaccines['Vaccines'] = vial_to_vaccs_dict[df_vaccines[add_cols[1]].value]
    # # Calculate vials using trays
    # trays = list(reversed_tray_dict.keys())
    # vials_lst = []
    # all_trays = []
    # for num in trays:
    #     print(df_grouped[num])
    #     num_trays = int(df_grouped[num][0])
    #     vials_lst.append(num * num_trays)
    #     all_trays.append(num_trays)

    # sum_vials = sum(vials_lst)
    # sum_vaccs = sum_vials * 6

    # Create calculated columns
    df_grouped['Sum Vials'] = sum_vials
    df_grouped['full trays'] = int(df_grouped[195])
    df_grouped['splitted trays'] = sum_trays - (df_grouped[195])
    df_grouped['Sum vials splitted'] = sum_vials - 195*int(df_grouped[195])
    df_grouped['Sum vaccinations'] = sum_vaccs
    df_grouped['isMissing'] = True in list(df_hosp[relevant_cols[1]].str.contains('חסר'))

    # Set the right order of columns
    df_grouped = df_grouped[final_cols]

    print(df_grouped.shape, sheet_name)
    print('--------------- DONE ----------------')
    return df_grouped


# Apply Function

In [20]:
dfs = {}

for sheet_num in range(1, len(xl_sheets)):
    try:
        sheet_name = xl_sheets[sheet_num]
        dfs[sheet_name] = validate_sheet(sheet_name)
    except Exception as e:
        print(f'There was an error!\nSheet Name: {sheet_name} \nERROR: ', e)

(1, 13) כללית
--------------- DONE ----------------
(1, 13) מכבי
--------------- DONE ----------------
(1, 13) מאוחדת
--------------- DONE ----------------
(1, 13) לאומית
--------------- DONE ----------------
(1, 13) צהל
--------------- DONE ----------------
(1, 13) נובולוג
--------------- DONE ----------------
There was an error!
Sheet Name: מלאי שסופק 
ERROR:  'קוד לקוח'


In [21]:
df_all = pd.concat(list(dfs.values()))
df_all

VialsPerTray,Entity,distribution points,Sum Vials,full trays,splitted trays,50,15,10,5,1,Sum vials splitted,Sum vaccinations,isMissing
0,כללית,70,1015,0,102,0,0,101,1,0,1015,10150,True
0,מכבי,47,500,0,50,0,0,50,0,0,500,5000,False
0,מאוחדת,60,760,0,76,0,0,76,0,0,760,7600,False
0,לאומית,16,160,0,16,0,0,16,0,0,160,1600,False
0,צהל,1,10,0,1,0,0,1,0,0,10,100,False
0,נובולוג,3,170,0,17,0,0,17,0,0,170,1700,False


### Export to Excel

In [22]:
df_all.to_excel(f'Summary-eilon-{XL_date}.xlsx', index=False)

In [23]:
print('---------- Process is DONE ----------')

---------- Process is DONE ----------


# Done

In [24]:
sheet_name = 'כללית'
# def validate_sheet(sheet_name):
# sheet_name = 'כללית'

# Get Excel sheet
df_hosp = pd.read_excel(path, sheet_name=sheet_name)
df_hosp
# Check if sheet is empty
if df_hosp.shape == (0, 0):
#     return None ############
        pass

# Calculate number of distribution points
POINTS = len(df_hosp[relevant_cols[1]].unique())
df_hosp[relevant_cols[1]] = df_hosp[relevant_cols[1]].apply(lambda x: str(x))

# Filter on relevant products
df_hosp_filtered = df_hosp[df_hosp[relevant_cols[0]].isin(tray_dict.keys())][relevant_cols]

# Change category name to numeric
# df_hosp_filtered[relevant_cols[0]] = df_hosp_filtered[relevant_cols[0]].apply(lambda x: tray_dict[x][0])
# df_hosp_filtered.sort_values([relevant_cols[0]], inplace=True)


# fill missing categories
all_vals = list(tray_dict.keys()) # get all possible values
sheet_vals = df_hosp_filtered[relevant_cols[0]].unique() # get all existing values in sheet
lst_addition = []

    # check for missing values
for i in all_vals:
    if i not in list(sheet_vals):
        lst_addition.append([i, 0, 0])

    # append fillers
df_hosp_filtered = pd.concat([df_hosp_filtered, pd.DataFrame(lst_addition, columns=df_hosp_filtered.columns)])


# Add column - Manufacturer
df_hosp_filtered['Manufacturer'] = df_hosp_filtered[relevant_cols[0]].apply(lambda x: tray_dict[x][1])
df_hosp_filtered['VialsPerTray'] = df_hosp_filtered[relevant_cols[0]].apply(lambda x: tray_dict[x][0])
add_cols = ['VialsPerTray', 'Manufacturer']


# # Group df by product name
df_grouped = df_hosp_filtered.groupby(add_cols).sum().reset_index()
df_grouped['Vials'] = df_grouped[relevant_cols[2]] * df_grouped[add_cols[0]]
# df_grouped = df_grouped.T.rename(columns=reversed_tray_dict) # pivot



df_vaccines = df_grouped.groupby(add_cols[1]).sum().reset_index()

def get_multiplier(row):
    return row[add_cols[1]]

df_vaccines['Vaccines'] = df_vaccines[add_cols[1]].apply(lambda x: vial_to_vaccs_dict[x]) * df_vaccines['Vials']

sum_vaccs = df_vaccines.Vaccines.sum()

df_grouped = df_grouped.groupby('VialsPerTray').sum()

sum_vials = df_grouped['Vials'].sum()

df_grouped = df_grouped.T.reset_index() # pivot
df_grouped = df_grouped[df_grouped['index'] == relevant_cols[-1]]

df_grouped['distribution points'] = POINTS
df_grouped['Entity'] = sheet_name

sum_trays = df_vaccines[relevant_cols[-1]].sum()
# df_vaccines['Vaccines'] = vial_to_vaccs_dict[df_vaccines[add_cols[1]].value]
# # Calculate vials using trays
# trays = list(reversed_tray_dict.keys())
# vials_lst = []
# all_trays = []
# for num in trays:
#     print(df_grouped[num])
#     num_trays = int(df_grouped[num][0])
#     vials_lst.append(num * num_trays)
#     all_trays.append(num_trays)

# sum_vials = sum(vials_lst)
# sum_vaccs = sum_vials * 6

# Create calculated columns
df_grouped['Sum Vials'] = sum_vials
df_grouped['full trays'] = int(df_grouped[195])
df_grouped['splitted trays'] = sum_trays - (df_grouped[195])
df_grouped['Sum vials splitted'] = sum_vials - 195*int(df_grouped[195])
df_grouped['Sum vaccinations'] = sum_vaccs
df_grouped['isMissing'] = True in list(df_hosp[relevant_cols[1]].str.contains('חסר'))

# Set the right order of columns
df_grouped = df_grouped[final_cols]

print(df_grouped.shape, sheet_name)
print('--------------- DONE ----------------')
# return df_grouped
df_grouped

(1, 13) כללית
--------------- DONE ----------------


VialsPerTray,Entity,distribution points,Sum Vials,full trays,splitted trays,50,15,10,5,1,Sum vials splitted,Sum vaccinations,isMissing
0,כללית,70,1015,0,102,0,0,101,1,0,1015,10150,True


In [26]:
df_grouped[195][0]

0

In [81]:
df_hosp_filtered

,תיאור פריט,קוד לקוח,"סה""כ כמות מארזים",Manufacturer,VialsPerTray
0,MODERNA COVID19 MRNA1273 VACCINE 5ML 29C 10VIAL,1049348,1,M,10
3,MODERNA COVID19 MRNA1273 VACCINE 5ML 29C 10VIAL,1046022,1,M,10
6,MODERNA COVID19 MRNA1273 VACCINE 5ML 29C 10VIAL,1046270,1,M,10
9,MODERNA COVID19 MRNA1273 VACCINE 5ML 29C 10VIAL,1046265,1,M,10
12,MODERNA COVID19 MRNA1273 VACCINE 5ML 29C 10VIAL,1046023,2,M,10
15,MODERNA COVID19 MRNA1273 VACCINE 5ML 29C 10VIAL,1046266,1,M,10
18,MODERNA COVID19 MRNA1273 VACCINE 5ML 29C 10VIAL,1046274,1,M,10
21,MODERNA COVID19 MRNA1273 VACCINE 5ML 29C 10VIAL,1049334,1,M,10
24,MODERNA COVID19 MRNA1273 VACCINE 5ML 29C 10VIAL,1045744,1,M,10
27,MODERNA COVID19 MRNA1273 VACCINE 5ML 29C 10VIAL,1046268,1,M,10
